# Creating a Customer Service Agent with Client-Side Tools
In this recipe, we'll demonstrate how to create a customer service chatbot using Claude 3 plus client-side tools. The chatbot will be able to look up customer information, retrieve order details, and cancel orders on behalf of the customer. We'll define the necessary tools and simulate synthetic responses to showcase the chatbot's capabilities.

## Step 1: Set up the environment
First, let's install the anthropic-rb gem.

In [ ]:
require "bundler/inline"

gemfile do
  source "https://rubygems.org"

  gem "anthropic-rb"
end

Then, we'll initialize the library with our API key.

In [2]:
require 'anthropic'

Anthropic.setup do |config|
  config.api_key = ENV.fetch('ANTHROPIC_API_KEY')
end

{:completions=>[{"version"=>"2023-06-01", "endpoint"=>"/v1/complete", "schema"=>{"type"=>"object", "required"=>["model", "prompt", "max_tokens_to_sample"], "properties"=>{"model"=>{"type"=>"string"}, "prompt"=>{"type"=>"string"}, "max_tokens_to_sample"=>{"type"=>"integer"}, "stop_sequences"=>{"type"=>"array", "items"=>{"type"=>"string"}}, "temperature"=>{"type"=>"number"}, "top_k"=>{"type"=>"integer"}, "top_p"=>{"type"=>"number"}, "metadata"=>{"type"=>"object"}, "stream"=>{"type"=>"boolean"}}, "additionalProperties"=>false}}], :messages=>[{"version"=>"2023-06-01", "endpoint"=>"/v1/messages", "schema"=>{"type"=>"object", "required"=>["model", "messages", "max_tokens"], "properties"=>{"model"=>{"type"=>"string"}, "messages"=>{"type"=>"array"}, "max_tokens"=>{"type"=>"integer"}, "system"=>{"type"=>"string"}, "stop_sequences"=>{"type"=>"array", "items"=>{"type"=>"string"}}, "temperature"=>{"type"=>"number"}, "top_k"=>{"type"=>"integer"}, "top_p"=>{"type"=>"number"}, "metadata"=>{"type"=>"o

## Step 2: Define the client-side tools
Next, we'll define the client-side tools that our chatbot will use to assist customers. We'll create three tools: get_customer_info, get_order_details, and cancel_order.

In [3]:
@tools = [
  {
    name: "get_customer_info",
    description: "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
    input_schema: {
      type: "object",
      properties: {
        customer_id: {
          type: "string",
          description: "The unique identifier for the customer."
        }
      },
      required: ["customer_id"]
    }
  },
  {
    name: "get_order_details",
    description: "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
    input_schema: {
      type: "object",
      properties: {
        order_id: {
          type: "string",
          description: "The unique identifier for the order."
        }
      },
      required: ["order_id"]
    }
  },
  {
    name: "cancel_order",
    description: "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
    input_schema: {
      type: "object",
      properties: {
        order_id: {
          type: "string",
          description: "The unique identifier for the order to be cancelled."
        }
      },
      required: ["order_id"]
    }
  }
]

[{:name=>"get_customer_info", :description=>"Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.", :input_schema=>{:type=>"object", :properties=>{:customer_id=>{:type=>"string", :description=>"The unique identifier for the customer."}}, :required=>["customer_id"]}}, {:name=>"get_order_details", :description=>"Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.", :input_schema=>{:type=>"object", :properties=>{:order_id=>{:type=>"string", :description=>"The unique identifier for the order."}}, :required=>["order_id"]}}, {:name=>"cancel_order", :description=>"Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.", :input_schema=>{:type=>"object", :properties=>{:order_id=>{:type=>"string", :description=>"The unique identifier for the order to be cancelled."}}, :required=>["order_id"]}}]

## Step 3: Simulate synthetic tool responses
Since we don't have real customer data or order information, we'll simulate synthetic responses for our tools. In a real-world scenario, these functions would interact with your actual customer database and order management system.

In [ ]:
def get_customer_info(customer_id)
  # Simulated customer data
  customers = {
    "C1" => { name: "John Doe", email: "john@example.com", phone: "123-456-7890" },
    "C2" => { name: "Jane Smith", email: "jane@example.com", phone: "987-654-3210" }
  }
  customers.fetch(customer_id, "Customer not found")
end

def get_order_details(order_id)
  # Simulated order data
  orders = {
    "O1" => { id: "O1", product: "Widget A", quantity: 2, price: 19.99, status: "Shipped" },
    "O2" => { id: "O2", product: "Gadget B", quantity: 1, price: 49.99, status: "Processing" }
  }
  orders.fetch(order_id, "Order not found")
end

def cancel_order(order_id)
  # Simulated order cancellation
  ["O1", "O2"].include?(order_id)
end

## Step 4: Process tool calls and return results
We'll create a function to process the tool calls made by Claude and return the appropriate results.

In [5]:
def process_tool_call(tool_name, tool_input)
  case tool_name
  when "get_customer_info"
    get_customer_info(tool_input["customer_id"])
  when "get_order_details"
    get_order_details(tool_input["order_id"])
  when "cancel_order"
    cancel_order(tool_input["order_id"])
  end
end

:process_tool_call

## Step 5: Interact with the chatbot
Now, let's create a function to interact with the chatbot. We'll send a user message, process any tool calls made by Claude, and return the final response to the user.

In [6]:
def chatbot_interaction(user_message)
  puts("#{'=' * 50}\nUser Message: #{user_message}\n#{'=' * 50}")
  message = Anthropic.messages(beta: 'tools-2024-04-04').create(
    model: 'claude-3-opus-20240229',
    max_tokens: 200,
    tools: @tools,
    messages: [{ role: 'user', content: user_message }]
  )
  puts("\nInitial Response:")
  puts("Stop Reason: #{message[:stop_reason]}")
  puts("Content: #{message[:content]}")

  if message[:stop_reason] == 'tool_use'
    tool_use_block = message[:content].find { |block| block[:type] == 'tool_use' }
    puts("\nTool Used: #{tool_use_block[:name]}")
    puts("Tool Input: #{tool_use_block[:input]}")
    tool_result = process_tool_call(tool_use_block[:name], tool_use_block[:input])     
    puts("Tool Result: #{tool_result}")
    response = Anthropic.messages(beta: 'tools-2024-04-04').create(
      model: 'claude-3-opus-20240229',
      max_tokens: 200,
      tools: @tools,
      messages: [
        { role: 'user', content: user_message },
        { role: 'assistant', content: message[:content] },
        {
          role: 'user',
          content: [
            {
              type: 'tool_result',
              tool_use_id: tool_use_block[:id],
              content: tool_result
            }
          ]
        }
      ]
    )
    puts("\nFinal Response: #{response[:content]}\n")
  end
end

:chatbot_interaction

## Step 6: Test the chatbot
Let's test our customer service chatbot with a few sample queries.

In [7]:
chatbot_interaction("Can you tell me the email address for customer C1?")
chatbot_interaction("What is the status of order O2?")
chatbot_interaction("Please cancel order O1 for me.")

User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [{:type=>"text", :text=>"<thinking>\nThe user is asking for the email address of a specific customer with the ID \"C1\". To retrieve this information, the get_customer_info function would be the most relevant tool, as it returns the customer's name, email, and phone number given a customer ID.\n\nThe get_customer_info function requires a single parameter:\n- customer_id (string): The customer ID is directly provided by the user as \"C1\".\n\nSince the required customer_id parameter is present in the user's request, we can proceed with calling the get_customer_info function.\n</thinking>"}, {:type=>"tool_use", :id=>"toolu_013hGH56hiMPa4CWMeicrNKg", :name=>"get_customer_info", :input=>{:customer_id=>"C1"}}]

Tool Used: get_customer_info
Tool Input: {:customer_id=>"C1"}
Tool Result: Customer not found

Final Response: [{:type=>"text", :text=>"Based on the function result, the

Anthropic::Errors::BadRequestError: {:type=>"error", :error=>{:type=>"invalid_request_error", :message=>"messages.2.content.0.tool_result.content: Found a boolean, but `tool_result` content must either be a string or a list of content blocks."}}